In [2]:
import pandas as pd
import seaborn as sb
import csv
from sympy import FiniteSet

import pysam
import os
import math
import random
import numpy as np

from pathlib import Path
from random import seed
from random import randint

In [3]:

def powerset(fullset):
  listsub = list(fullset)
  subsets = []
  for i in range(2**len(listsub)):
    subset = []
    for k in range(len(listsub)):            
      if i & 1<<k:
        subset.append(listsub[k])
    subsets.append(subset)        
  return subsets

In [4]:
def Sy(dic,X):
    dict_items=dic.items()
    i=1
    for k in X:
        myValue=k
        globals()["Y{}".format(i)] =[key for key,value in dict_items if value==myValue]
        mykey=[key for key,value in dict_items if value==myValue]
        i+=1
    return Y1,Y2,Y3,Y4,Y5,Y6,Y7


In [5]:
def depth_Y(conjunto,A):
    d1 = pd.read_csv(ruta_depth + conjunto+ '-Simulados-subsp_capsici_1101_sorted.txt', sep='\t', header=None) #importar las tablas para cada genoma
    d2 = pd.read_csv(ruta_depth + conjunto+ '-Simulados-subsp_nebraskensis_61-1_sorted.txt', sep='\t', header=None)
    d3 = pd.read_csv(ruta_depth + conjunto+ '-Simulados-subsp_michi_contigs_sorted.txt', sep='\t', header=None) 
    a1=d1[2].sum()
    a2=d1[2].sum()
    a3=d1[2].sum()
    ay=(a1+a2+a3)/3
    l1=len(d1)
    l2=len(d2)
    l3=len(d3)
    m=len(A)
    dyx1=ay/(l1)*1/m**2
    dyx2=ay/(l2)*1/m**2
    dyx3=ay/(l3)*1/m**2
    return dyx1,dyx2,dyx3

In [10]:
folder='1TP' #nombre donde esta los genomas
folder_bash='1TP'

In [8]:
%%bash
ftem=$folder_bash
echo $ftem

In [33]:
os.makedirs('output-tda/'+folder)
os.makedirs('output-tda/'+folder+'/index-bowtie')
os.makedirs('output-tda/'+folder+'/out-bowtie')
os.makedirs('output-tda/'+folder+'/bam')
os.makedirs('output-tda/'+folder+'/sorted')
os.makedirs('output-tda/'+folder+'/tablas')
os.makedirs('output-tda/'+folder+'/SyReads')
os.makedirs('output-tda/'+folder+'/sam_extract')
os.makedirs('output-tda/'+folder+'/depth')

In [ ]:

%%bash 
source /opt/anaconda3/etc/profile.d/conda.sh
conda activate metagenomics
ls clavi_select/$folder/*fna | while read line; 
do 
    id=$(echo $line| rev| cut -d'_' -f1,2,3 |rev |cut -d'.' -f1);
    bowtie2-build $line output-tda/$folder/index-bowtie/$id;
    echo "$id" >>list-genomasrefencia2TP.txt
done

In [ ]:
%%bash 
source /opt/anaconda3/etc/profile.d/conda.sh
conda activate metagenomics

cat list-genomasrefencia3TP.txt | while read line;
do
bowtie2 -x output-tda/3TP/index-bowtie/$line -U ReadsSimulados/reads-$folder.fastq --no-unal -p 12 -S output-tda/$folder/out-bowtie/Simulados-${line}.sam
done

6000 reads; of these:
  6000 (100.00%) were unpaired; of these:
    2657 (44.28%) aligned 0 times
    3286 (54.77%) aligned exactly 1 time
    57 (0.95%) aligned >1 times
55.72% overall alignment rate
6000 reads; of these:
  6000 (100.00%) were unpaired; of these:
    1555 (25.92%) aligned 0 times
    4220 (70.33%) aligned exactly 1 time
    225 (3.75%) aligned >1 times
74.08% overall alignment rate
6000 reads; of these:
  6000 (100.00%) were unpaired; of these:
    3751 (62.52%) aligned 0 times
    2236 (37.27%) aligned exactly 1 time
    13 (0.22%) aligned >1 times
37.48% overall alignment rate

In [ ]:
%%bash
for infile in output-tda/$folder/out-bowtie/*.sam
do 
	base=$(basename ${infile} .sam)
	samtools view ${infile} -o output-tda/$folder/bam/${base}.bam
    samtools sort ${infile} -o output-tda/$folder/sorted/${base}_sorted.sam
done

In [ ]:
%%bash
for infile in output-tda/$folder/bam/*.bam
do 
	base=$(basename ${infile} .bam)
	samtools sort ${infile} -o output-tda/$folder/sorted/${base}_sorted.bam
	samtools view ${infile} > output-tda/$folder/bam/${base}.txt
done

In [ ]:
%%bash
cd ~/GIT/clavibacter/scripts-tda/
ls output-tda/$folder/bam/*.txt | while read line
do
    base=$(basename ${line} .txt)
    cut -f 1-6 $line > output-tda/$folder/tablas/${base}_extrac.txt
done

In [11]:
ruta_tablas = '/home/shaday/GIT/clavibacter/scripts-tda/output-tda/'+folder+'/tablas/'
contenido = os.listdir(ruta_tablas)
genomes = []
df=[]
for i in contenido:
    dftt = pd.read_csv(ruta_tablas + i, sep='\t', header=None) #importar las tablas para cada genoma
    dftt['Organims']=i[16:-11] #agregar el nombre del organismo
    df.append(dftt) #agregar al dataframe
    genomes.append(i[16:-11])
    
df=pd.concat(df)
df.columns= ['Qname','Flag', 'Rname', 'Pos','MapQ','CIGAR','Organims'] #cambiar nombre a las columnas del dataframe
df 

,Qname,Flag,Rname,Pos,MapQ,CIGAR,Organims
0,NZ_CP048050.1_0,0,NZ_CP048050.1,98092,42,150M,capsici_1101
1,NC_009480.1_0,0,NZ_CP048050.1,68359,42,150M,capsici_1101
2,NC_009479.1_0,0,NZ_CP048050.1,16948,42,150M,capsici_1101
3,NZ_CP048050.1_1,0,NZ_CP048050.1,93115,23,150M,capsici_1101
4,NC_009480.1_1,0,NZ_CP048050.1,48066,42,150M,capsici_1101
...,...,...,...,...,...,...,...
4443,NC_009480.1_985,0,NC_009480.1,168423,7,150M,michi_contigs
4444,NC_009478.1_985,0,NC_009480.1,3119250,42,150M,michi_contigs
4445,NZ_CP033723.2_985,0,NC_009478.1,1232,42,150M,michi_contigs
4446,NZ_CP048049.1_986,0,NC_009479.1,5287,42,150M,michi_contigs


In [12]:
X=df['Organims'].unique().tolist()
len(X)
dic_Organisms={}
for k in range(len(X)):
    dic_Organisms[X[k]]="X{0}".format(k+1)
dic_Organisms
df2=df.replace({"Organims": dic_Organisms}) 

X2=df2['Organims'].unique().tolist() #lista de genomas 
A=powerset(X2) #conjunto potencia
A=A[1:] #eliminiamos el conjunto vacio
df2

,Qname,Flag,Rname,Pos,MapQ,CIGAR,Organims
0,NZ_CP048050.1_0,0,NZ_CP048050.1,98092,42,150M,X1
1,NC_009480.1_0,0,NZ_CP048050.1,68359,42,150M,X1
2,NC_009479.1_0,0,NZ_CP048050.1,16948,42,150M,X1
3,NZ_CP048050.1_1,0,NZ_CP048050.1,93115,23,150M,X1
4,NC_009480.1_1,0,NZ_CP048050.1,48066,42,150M,X1
...,...,...,...,...,...,...,...
4443,NC_009480.1_985,0,NC_009480.1,168423,7,150M,X3
4444,NC_009478.1_985,0,NC_009480.1,3119250,42,150M,X3
4445,NZ_CP033723.2_985,0,NC_009478.1,1232,42,150M,X3
4446,NZ_CP048049.1_986,0,NC_009479.1,5287,42,150M,X3


In [13]:
df2.to_csv('MapeoReads-1TP.csv')

In [14]:
dic={}
l=0
Reads=df2['Qname'].unique().tolist()
for k in Reads:
    valores=list(df2.loc[df2.Qname==k].iloc[:,6])
    dic[k]=valores


In [33]:
dic["NZ_CP048050.1_0"]

['X1']

In [28]:
dic_Organisms


{'capsici_1101': 'X1', 'nebraskensis_61-1': 'X2', 'michi_contigs': 'X3'}

In [34]:
A

[['X1'],
 ['X2'],
 ['X1', 'X2'],
 ['X3'],
 ['X1', 'X3'],
 ['X2', 'X3'],
 ['X1', 'X2', 'X3']]

In [29]:
# Calculamos SY para cada Y en 2**X
SY=[]
SY= Sy(dic,A)

In [30]:
A

[['X1'],
 ['X2'],
 ['X1', 'X2'],
 ['X3'],
 ['X1', 'X3'],
 ['X2', 'X3'],
 ['X1', 'X2', 'X3']]

In [39]:
#Exportamos Sy a txt
for k in range(len(SY)):
    with open('output-tda/'+folder+'/Seads/Sy{0}.txt'.format(k), 'w') as f:
        for line in SY[k]:
            f.write(f"{line}\n")

In [ ]:
%%bash 
for line in output-tda/$folder/SyReads/*.txt 
do
    bash 02-extract-reads-for-SY2.sh $line $folder
done

In [ ]:
for line in output-tda/$folder/sam_extract/*.bam
do
    base=$(basename ${line} .bam)
    samtools depth ${line} > output-tda/$folder/depth/${base}.txt
done

In [40]:
ruta_depth = '/home/shaday/GIT/clavibacter/scripts-tda/output-tda/'+folder+'/depth/'
contenido_depth = os.listdir(ruta_depth)
contenido_depth

['Sy3-Simulados-subsp_capsici_1101_sorted.txt',
 'Sy4-Simulados-subsp_michi_contigs_sorted.txt',
 'Sy6-Simulados-subsp_michi_contigs_sorted.txt',
 'Sy5-Simulados-subsp_nebraskensis_61-1_sorted.txt',
 'Sy2-Simulados-subsp_nebraskensis_61-1_sorted.txt',
 'Sy1-Simulados-subsp_capsici_1101_sorted.txt',
 'Sy0-Simulados-subsp_michi_contigs_sorted.txt',
 'Sy6-Simulados-subsp_capsici_1101_sorted.txt',
 'Sy3-Simulados-subsp_nebraskensis_61-1_sorted.txt',
 'Sy2-Simulados-subsp_michi_contigs_sorted.txt',
 'Sy1-Simulados-subsp_michi_contigs_sorted.txt',
 'Sy5-Simulados-subsp_capsici_1101_sorted.txt',
 'Sy2-Simulados-subsp_capsici_1101_sorted.txt',
 'Sy6-Simulados-subsp_nebraskensis_61-1_sorted.txt',
 'Sy4-Simulados-subsp_capsici_1101_sorted.txt',
 'Sy3-Simulados-subsp_michi_contigs_sorted.txt',
 'Sy5-Simulados-subsp_michi_contigs_sorted.txt',
 'Sy0-Simulados-subsp_capsici_1101_sorted.txt',
 'Sy4-Simulados-subsp_nebraskensis_61-1_sorted.txt',
 'Sy0-Simulados-subsp_nebraskensis_61-1_sorted.txt',
 'S

In [41]:
dep=[]
for i in range(len(A)):
    globals()["Dep_Y{}".format(i)]=depth_Y('Sy{0}'.format(i),A[i])
    dep.append(depth_Y('Sy{0}'.format(i),A[i]))

In [42]:
dp = pd.DataFrame(dep,columns=dic_Organisms.values())
dp['Y']=A
dp

,X1,X2,X3,Y
0,1.157930,1.530690,1.064423,[X1]
1,1.030891,1.532315,0.887414,[X2]
2,0.251537,0.341087,0.207718,"[X1, X2]"
3,1.190809,1.523483,1.100109,[X3]
4,0.262426,0.365382,0.225751,"[X1, X3]"
5,0.265661,0.356367,0.221977,"[X2, X3]"
6,0.129057,0.175216,0.116525,"[X1, X2, X3]"


In [43]:
dp.to_csv('Depth1TP.csv')

In [1]:
import pysam